## Sommaire
- [Importations](#importations)
- [Extraction de la data](#lecture-et-extraction-de-la-data-du-fichier-msd_summary_fileh5)

### Lexique :
1. **analysis_sample_rate :** Le taux d'échantillonnage utilisé pour l'analyse du morceau audio.
2. **duration :** La durée du morceau en unités de temps (secondes, millisecondes, etc.).
3. **track_id :** Un identifiant unique pour le morceau.
4. **danceability :** Une mesure de la danseabilité du morceau, qui indique à quel point il est adapté à la danse.
5. **loudness :** L'intensité sonore globale du morceau en décibels (dB).
6. **tempo :** Le tempo du morceau, exprimé en battements par minute (BPM).
7. **start_of_fade_out :** Le point de départ du fondu de sortie du morceau.
8. **end_of_fade_in :** Le point de fin du fondu d'entrée du morceau.
9. **song_id :** Un identifiant unique pour la chanson.
10. **key :** La tonalité principale de la chanson (par exemple, C, D, E, etc.).
11. **key_confidence :** La confiance dans la détermination de la tonalité.
12. **time_signature :** La signature rythmique de la chanson (par exemple, 4/4, 3/4, etc.).
13. **genres :** Les genres musicaux associés à la chanson. C'est le champ que vous cherchez à prédire avec votre modèle.

___

##### Importations :

In [ ]:
import os, h5py
import pandas as pd

##### Lecture et extraction de la data du fichier msd_summary_file.h5

In [ ]:
directory_path = './Data/'
data_list = []

for root, dirnames, filenames in os.walk(directory_path):
    for filename in filenames:
        if filename.endswith('.h5'):
            hdf_file_path = os.path.join(root, filename)
            hdf = h5py.File(hdf_file_path, 'r')

            analysis_data = hdf['/analysis/songs/'][:]
            metadata_data = hdf['/metadata/songs/'][:]
            musicbrainz_data = hdf['/musicbrainz/songs/'][:]

            combined_data = pd.concat([pd.DataFrame(analysis_data), pd.DataFrame(metadata_data), pd.DataFrame(musicbrainz_data)], axis=1)
            
            data_list.append(combined_data)
            
            hdf.close()

final_data = pd.concat(data_list, ignore_index=True)

In [ ]:
final_data.head()

In [ ]:
# Convertir les données en chaînes de caractères
final_data['audio_md5'] = final_data['audio_md5'].str.decode('utf-8')
final_data['release'] = final_data['release'].str.decode('utf-8')
final_data['song_id'] = final_data['song_id'].str.decode('utf-8')
final_data['title'] = final_data['title'].str.decode('utf-8')
final_data['artist_name'] = final_data['artist_name'].str.decode('utf-8')
final_data['artist_id'] = final_data['artist_id'].str.decode('utf-8')
final_data['track_id'] = final_data['track_id'].str.decode('utf-8')
final_data['genre'] = final_data['genre'].str.decode('utf-8')

In [ ]:
# Supprimer les caractères "b" au début de chaque élément
final_data['audio_md5'] = final_data['audio_md5'].str.replace('^b', '', regex=True)
final_data['release'] = final_data['release'].str.replace('^b', '', regex=True)
final_data['song_id'] = final_data['song_id'].str.replace('^b', '', regex=True)
final_data['title'] = final_data['title'].str.replace('^b', '', regex=True)
final_data['artist_name'] = final_data['artist_name'].str.replace('^b', '', regex=True)
final_data['artist_id'] = final_data['artist_id'].str.replace('^b', '', regex=True)
final_data['genre'] = final_data['genre'].str.replace('^b', '', regex=True)
# final_data['track_id'] = final_data['track_id'].str.replace('^b', '', regex=True)

### Filtrage des colonnes, pour ne garder que celles-qui sont intéressantes

In [ ]:
# Je filtre les colonnes les plus pertinantes
columns_to_filter= ['Column1', 'analysis_sample_rate', 'duration', 'track_id', 'genre', 'danceability', 'loudness', 'tempo','start_of_fade_out','end_of_fade_in','song_id','key','key_confidence','time_signature']

In [ ]:
final_data = final_data.filter(columns_to_filter)
final_data = final_data.drop('genre', axis=1)

In [ ]:
final_data.head()

In [ ]:
final_data.to_csv('./Data/final_data.csv', sep=',', encoding='utf-8', index=False)

In [ ]:
genres = pd.read_csv('./Data/genre_cls_beatunes.csv', sep=',', encoding='utf-8')

In [ ]:
genres.head()

In [ ]:
# Permet de renommer la colonne
genres = genres.rename(columns={'seed-genre': 'genres'})

In [ ]:
final_csv = pd.merge(final_data, genres, on='track_id')

In [ ]:
final_csv.to_csv('./Data/1Mdata.csv', sep=',', encoding='utf-8', index=False)

In [ ]:
final_csv.head()

In [ ]:
_10Kdata_with_genre = pd.read_csv('./Data/10Kdata_with_genre.csv', sep=',', encoding='utf-8')

- [Retour au lexique](#lexique)
- [Filtrage des colonnes](#filtrage-des-colonnes-pour-ne-garder-que-celles-qui-sont-intéressantes)

In [ ]:
# Apercu rapide du dataset avec les genres
_10Kdata_with_genre.head()